# Multithreading

In [7]:
# without thread

import time

def calc_square(numbers):
    for n in numbers:
        time.sleep(0.5)
        print('square:',n**2)
        

def calc_cube(numbers):
    for n in numbers:
        time.sleep(0.5)
        print('cube:',n**3)
        

arr=[2,3,4,5,6]

t=time.time()

calc_square(arr)
calc_cube(arr)

print('Program completed in' + str(time.time()-t))
        
        
        
        

square: 4
square: 9
square: 16
square: 25
square: 36
cube: 8
cube: 27
cube: 64
cube: 125
cube: 216
Program completed in5.009404182434082


In [11]:
# with thread

import time
import threading

def calc_square(numbers):
    for n in numbers:
        time.sleep(0.5)
        print('square:',n**2)
        

def calc_cube(numbers):
    for n in numbers:
        time.sleep(0.5)
        print('cube:',n**3)
        
arr=[2,3,4,5,6]

t1=threading.Thread(target=calc_square, args=(arr,))
t2=threading.Thread(target=calc_cube, args=(arr,))

t=time.time()  
t1.start()
t2.start()

t1.join()
t2.join()

print('Program completed in' + str(time.time()-t))

square: 4
cube: 8
square: 9
cube: 27
square: 16
cube: 64
square: 25
cube: 125
square: 36
cube: 216
Program completed in2.5066347122192383


Main thread must wait until the newly created threads terminated. join() function server this purpose.

In [12]:
# Threads share the same memory address.

import time
import threading

result=[]

def calc_square(numbers):
    global result
    for n in numbers:
        time.sleep(0.5)
        #print('square:',n**2)
        result.append(n**2)
        

def calc_cube(numbers):
    global result
    for n in numbers:
        time.sleep(0.5)
        #print('cube:',n**3)
        result.append(n**3)
        
arr=[2,3,4,5,6]

t1=threading.Thread(target=calc_square, args=(arr,))
t2=threading.Thread(target=calc_cube, args=(arr,))

t=time.time()  
t1.start()
t2.start()

t1.join()
t2.join()

print('result:', result)
print('Program completed in' + str(time.time()-t))

result: [4, 8, 9, 27, 16, 64, 25, 125, 216, 36]
Program completed in2.503723382949829


# Multiprocessing

The major difference between the two is that in multithreading threads are being executed in one process sharing common address space whereas in multi processing different processes have different address space. Thus creating multiple processes is costly compare to threads.

In [22]:
# multiple process without shared memory

import time
import multiprocessing

result=[]
def calc_square(numbers):
    for n in numbers:
        time.sleep(0.5)
        #print('square:',n**2)
        result.append(n**2)
    print('square result:', result)
   
        
def calc_cube(numbers):
    for n in numbers:
        time.sleep(0.5)
        #print('cube:',n**3) 
        result.append(n**3)
    print('cube result:', result) 

if __name__ == '__main__':
    arr=[2,3,4,5,6]
    p1=multiprocessing.Process(target=calc_square,args=(arr,))
    p2=multiprocessing.Process(target=calc_cube,args=(arr,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('result:', result)
    print('Done!')
    
    
    
    

square result: [4, 9, 16, 25, 36]
cube result: [8, 27, 64, 125, 216]
result: []
Done!


Every process has its own address space. Thus program variables are not shared between two processes. You need to use interprocess communication (IPC) techniques if we want to share data between two processes.

In [46]:
# multiple processes with shared memory. Shared memory resides outside process and been shared by new created
# process as well as main process. There are two ways shared memory can be accessed. Arrays and Values.

import multiprocessing


def calc_square(numbers, result, val):
    for idx, n in enumerate(numbers):
        result[idx]= n**2
    print('square result:', result[:])
    #print(type(val))
    val.value=6.0
    print(val.value)
    
    
if __name__ == '__main__':
    numbers=[3,6,9]
    result=multiprocessing.Array('i',3)  # result is shared Variable. i denotes data type. here it is integer. 3 is size
    val=multiprocessing.Value('d',0.0)
    p=multiprocessing.Process(target=calc_square, args=(numbers, result,val))
    
    p.start()
    p.join()
    
    print('square result outside process:', result[:])
    print(val.value)
    
    

square result: [9, 36, 81]
6.0
square result outside process: [9, 36, 81]
6.0


# Multiprocessng Queue

In [47]:
# multiprocessing queue lives in shared memory

import multiprocessing


def calc_square(numbers, q):
    for n in numbers:
        q.put(n**2)
        
    
if __name__ == '__main__':
    numbers=[3,6,9]
    q= multiprocessing.Queue()
    p=multiprocessing.Process(target=calc_square, args=(numbers,q))
    
    p.start()
    p.join()

    while q.empty() is False:
        print(q.get())
    
    

9
36
81


# Lock

In [102]:
import time
import multiprocessing

def deposit(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value + 1
        lock.release()

def withdraw(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value - 1
        lock.release()

if __name__ == '__main__':
    balance = multiprocessing.Value('i', 200)
    lock = multiprocessing.Lock()
    d = multiprocessing.Process(target=deposit, args=(balance,lock))
    w = multiprocessing.Process(target=withdraw, args=(balance,lock))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

200


# Multiprocessing Pool (Map Reduce)

In [103]:
def func(num):
    return num*num

arr=[2,3,4,5]
result=[]

for n in arr:
    result.append(func(n))
    

print(result)    

[4, 9, 16, 25]


In [107]:
from multiprocessing import Pool

def func(num):
    return num*num


arr=[2,3,4,5]
p=Pool()
result=p.map(func,arr)

p.close()
p.join()

print(result)    

[4, 9, 16, 25]


In [116]:
from multiprocessing import Pool

def func_map(num):
    return num*num


p=Pool()
result=p.map(func,[i for i in range(1000)])

p.close()
p.join()

print(result)   

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [115]:
[i for i in range(100)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]